Utils

In [1]:
import logging
import os
from pyspark.sql import SparkSession


def logging_process(log_file="script/log/info.log"):
    # Configure logging
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(
        filename=log_file,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s"
    )
    logger = logging.getLogger()
    return logger

def load_log_msg(spark: SparkSession, log_msg):

    DB_URL = "jdbc:postgresql://pipeline_db:5432/etl_log"
    table_name = "etl_log"

    # set config
    connection_properties = {
        "user":"postgres",
        "password":"cobapassword",
        "driver": "org.postgresql.Driver"
    }

    log_msg.write.jdbc(url = DB_URL,
                  table = table_name,
                  mode = "append",
                  properties = connection_properties)


def init_spark_session():
    spark = SparkSession.builder.appName(
        "Exercise Data Pipeline Week_6"
    ).getOrCreate()

    # handle legacy time parser
    spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

    return spark

staging_extract

In [2]:
from utils.helper import logging_process, init_spark_session, load_log_msg
from datetime import datetime
import logging
import pyspark

logging_process()


def extract_data(data_name: str, format_data: str) -> pyspark.sql.DataFrame:
    spark = init_spark_session()

    DB_URL = "jdbc:postgresql://source_db:5432/startup_investments"
    DB_USER = "postgres"
    DB_PASS = "cobapassword"

    jdbc_url = DB_URL
    connection_properties = {
        "user": DB_USER,
        "password": DB_PASS,
        "driver": "org.postgresql.Driver"
    }

    current_timestamp = datetime.now()
    log_message = None

    try:
        logging.info(f"===== Start Extracting {data_name} data =====")

        if format_data.lower() == "csv":
            df = spark.read.csv(f"data/{data_name}.csv", header=True)

        elif format_data.lower() == "db":
            df = spark.read.jdbc(
                url=jdbc_url,
                table=data_name,
                properties=connection_properties
            )

        else:
            raise ValueError("Format data not supported yet")

        logging.info(f"===== Finish Extracting {data_name} data =====")

        # Log success
        log_message = spark.sparkContext.parallelize([(
            "sources", "extract", "success", format_data, data_name, current_timestamp
        )]).toDF(["step", "process", "status", "source", "table_name", "etl_date"])

        return df

    except Exception as e:
        logging.error("====== Failed to Extract Data ======")
        logging.error(str(e))

        # Log failure
        log_message = spark.sparkContext.parallelize([(
            "sources", "extraction", "failed", format_data, data_name, current_timestamp, str(e)
        )]).toDF(["step", "process", "status", "source", "table_name", "etl_date", "error_msg"])

        raise

    finally:
        if log_message:
            try:
                load_log_msg(spark, log_message)
            except Exception as log_err:
                logging.error(f"Failed to write log to DB: {log_err}")


staging_load

In [3]:
from utils.helper import logging_process, init_spark_session, load_log_msg
import logging
import pyspark
from sqlalchemy import create_engine, text
from datetime import datetime

logging_process()


def load_data(df_result: pyspark.sql.DataFrame, table_name: str) -> None:
    """
    Function to truncate a table using SQLAlchemy and then load data into it using PySpark,
    while logging success/failure ETL events into a log table.
    """

    logging_process()

    spark = init_spark_session()
    current_timestamp = datetime.now()
    log_message = None

    # DB config
    DB_HOST = "pipeline_db"
    DB_PORT = "5432"
    DB_NAME = "staging"
    DB_USER = "postgres"
    DB_PASS = "cobapassword"

    jdbc_url = f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}"
    sqlalchemy_url = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

    connection_properties = {
        "user": DB_USER,
        "password": DB_PASS,
        "driver": "org.postgresql.Driver"
    }

    try:
        logging.info("===== Start Load data to the database =====")

        # TRUNCATE TABLE using SQLAlchemy
        engine = create_engine(sqlalchemy_url)
        with engine.connect() as connection:
            connection.execute(text(f"TRUNCATE TABLE {table_name} CASCADE"))
            logging.info(f"===== Truncated table {table_name} successfully =====")

        # Load data using PySpark
        df_result.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="append",
            properties=connection_properties,
        )

        logging.info("===== Finish Load data to the database =====")

        # SUCCESS log
        log_message = spark.sparkContext.parallelize([(
            "targets", "load", "success", "db", table_name, current_timestamp
        )]).toDF(["step", "process", "status", "source", "table_name", "etl_date"])

    except Exception as e:
        logging.error("===== Failed Load data to the database =====")
        logging.error(str(e))

        # FAILURE log
        log_message = spark.sparkContext.parallelize([(
            "targets", "load", "failed", "db", table_name, current_timestamp, str(e)
        )]).toDF(["step", "process", "status", "source", "table_name", "etl_date", "error_msg"])

        raise

    finally:
        if log_message is not None:
            try:
                load_log_msg(spark, log_message)
                logging.info("ETL log inserted successfully")
            except Exception as log_err:
                logging.error(f"Failed to write log to DB: {log_err}")


staging_pipeline

In [6]:
import logging

# Initialize logging
logging_process()

if __name__ == "__main__":
    logging.info("===== Start Banking Data Pipeline =====")

    try:
        # Extract data from CSV and database
        df_people = extract_data(data_name="people", format_data="csv")
        df_relationships = extract_data(data_name="relationships", format_data="csv")
        df_acquisition = extract_data(data_name="acquisition", format_data="db")
        df_funds = extract_data(data_name="funds", format_data="db")
        df_funding_rounds = extract_data(data_name="funding_rounds", format_data="db")
        df_company = extract_data(data_name="company", format_data="db")
        df_investments = extract_data(data_name="investments", format_data="db")
        df_ipos = extract_data(data_name="ipos", format_data="db")

        # Transform each dataset separately
        # df_transactions = transform_data(df_transactions, "transactions")

        # Load each transformed dataset into the data warehouse
        load_data(df_people, table_name="people")
        load_data(df_relationships, table_name="relationships")
        load_data(df_acquisition, table_name="acquisition")
        load_data(df_funds, table_name="funds")
        load_data(df_funding_rounds, table_name="funding_rounds")
        load_data(df_company, table_name="company")
        load_data(df_investments, table_name="investments")
        load_data(df_ipos, table_name="ipos")

        logging.info("===== Finish Investment Data Pipeline =====")

    except Exception as e:
        logging.error("===== Data Pipeline Failed =====")
        logging.error(e)
        raise

In [18]:
        load_data(df_people, table_name="people")
        load_data(df_relationships, table_name="relationships")
        load_data(df_acquisition, table_name="acquisition")
        load_data(df_funds, table_name="funds")
        load_data(df_funding_rounds, table_name="funding_rounds")
        load_data(df_company, table_name="company")
        load_data(df_investments, table_name="investments")
        load_data(df_ipos, table_name="ipos")

In [22]:
df_ipos = extract_data(data_name="ipos", format_data="db")

In [23]:
load_data(df_ipos, table_name="ipos")

Profiling

In [26]:
import pandas as pd
from datetime import datetime

def profile_dataframe(df: pd.DataFrame) -> dict:
    report = {}
    for col in df.columns:
        data_type = str(df[col].dtype)
        missing_percentage = df[col].isnull().mean() * 100
        entry = {"data_type": data_type}

        if missing_percentage > 0:
            entry["percentage_missing_value"] = round(missing_percentage, 2)
        else:
            entry["percentage_missing_value"] = 0.0

        if data_type == 'object' or data_type.startswith('str'):
            unique_values = df[col].dropna().unique()
            if len(unique_values) <= 10:
                entry["unique_value"] = unique_values.tolist()
        
        if "date" in col or "at" in col.lower():
            try:
                valid_dates = pd.to_datetime(df[col], errors='coerce')
                percentage_valid_date = valid_dates.notnull().mean() * 100
                entry["percentage_valid_date"] = round(percentage_valid_date, 2)
            except Exception:
                entry["percentage_valid_date"] = 0.0

        report[col] = entry

    return {
        "created_at": datetime.now().strftime("%Y-%m-%d"),
        "report": report
    }

# Run the profiling
profile_report = profile_dataframe(people_df)
print(profile_report)


TypeError: 'Column' object is not callable

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count
from utils.helper import init_spark_session
from datetime import datetime
import json

def profile_spark(df: pd.DataFrame) -> dict:
    spark = init_spark_session()
    report = {}
    total_rows = df.count()

    for col_name in df.columns:
        data_type = str(df.schema[col_name].dataType)
        
        # Hitung missing value (null atau kosong)
        missing_count = df.filter(col(col_name).isNull() | (col(col_name) == "")).count()
        missing_percentage = (missing_count / total_rows) * 100 if total_rows > 0 else 0

        entry = {
            "data_type": data_type,
            "percentage_missing_value": round(missing_percentage, 2)
        }

        # Hitung nilai unik (jika tidak terlalu banyak)
        unique_values = df.select(col_name).distinct().limit(5).rdd.flatMap(lambda x: x).collect()
        entry["unique_value"] = unique_values

        # Deteksi valid date
        if "date" in col_name.lower() or "at" in col_name.lower():
            try:
                valid_dates = df.withColumn("parsed", col(col_name).cast("timestamp"))
                valid_count = valid_dates.filter(col("parsed").isNotNull()).count()
                entry["percentage_valid_date"] = round((valid_count / total_rows) * 100, 2) if total_rows > 0 else 0
            except:
                entry["percentage_valid_date"] = 0.0

        report[col_name] = entry

    profile_result = {
        "person_in_charge": "Hudiya Resa",
        "created_at": datetime.now().strftime("%Y-%m-%d"),
        "report": report
    }

    # take variable name in df
    df_name = [name for name, value in globals().items() if value is df][0]

    # Save as JSON
    os.makedirs("profiling/output", exist_ok=True)
    file_path = f"profiling/output/{df_name}_profile_{datetime.now().strftime('%Y%m%d')}.json"
    with open(file_path, "w") as f:
        json.dump(profile_result, f, indent=4, default=str)

    print(f"Profiling report saved to: {file_path}")
    return profile_result

In [53]:
from utils.helper import init_spark_session
from staging.extract.extract_data import extract_data

spark = init_spark_session()

people_df = extract_data(data_name="people", format_data="csv")
profile_report = profile_spark(people_df)
print(profile_report)

Profiling report saved to: profiling/output/people_df_profile_20250423.json
{'person_in_charge': 'Hudiya Resa', 'created_at': '2025-04-23', 'report': {'people_id': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['296', '467', '675', '691', '829']}, 'object_id': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['p:73', 'p:171', 'p:214', 'p:486', 'p:625']}, 'first_name': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['Hideki', 'Tyler', 'Rony', 'Laurence', 'Lorne']}, 'last_name': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['Nerst', 'Banerjee', 'Cancel', 'Rekhi', 'Silverstein']}, 'birthplace': {'data_type': 'StringType()', 'percentage_missing_value': 87.61, 'unique_value': ['Gainesville, FL', 'Brisbane, Australia', 'Utah', 'Antwerp', 'Manilla']}, 'affiliation_name': {'data_type': 'StringType()', 'percentage_missing_value': 0.01, 'unique_value': ['Blue Nile', 'P

In [54]:
profile_report = profile_spark(df_relationships)
print(profile_report)

Profiling report saved to: profiling/output/df_relationships_profile_20250423.json
{'person_in_charge': 'Hudiya Resa', 'created_at': '2025-04-23', 'report': {'relationship_id': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['675', '829', '1090', '1159', '1436'], 'percentage_valid_date': 99.41}, 'person_object_id': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['p:73', 'p:171', 'p:214', 'p:486', 'p:683']}, 'relationship_object_id': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['c:933', 'c:2009', 'c:396', 'c:3146', 'c:3173'], 'percentage_valid_date': 0.0}, 'start_at': {'data_type': 'StringType()', 'percentage_missing_value': 53.48, 'unique_value': ['2010-10-01 00:00:00.000', '2010-05-24 00:00:00.000', '2006-09-19 00:00:00.000', '2006-04-15 00:00:00.000', '1969-01-01 00:00:00.000'], 'percentage_valid_date': 46.52}, 'end_at': {'data_type': 'StringType()', 'percentage_missing_value': 85.71, '

In [62]:
profile_report = profile_spark(df_company)
print(profile_report)

Profiling report saved to: profiling/output/df_company_profile_20250423.json
{'person_in_charge': 'Hudiya Resa', 'created_at': '2025-04-23', 'report': {'office_id': {'data_type': 'IntegerType()', 'percentage_missing_value': 0.0, 'unique_value': [148, 463, 471, 833, 1238]}, 'object_id': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['c:396', 'c:545', 'c:636', 'c:933', 'c:1558']}, 'description': {'data_type': 'StringType()', 'percentage_missing_value': 41.73, 'unique_value': ['Corporate Headquarters', 'Technology & Life Science', 'Bangalore', 'Cynapse Headquarters', 'HQ office']}, 'region': {'data_type': 'StringType()', 'percentage_missing_value': 0.0, 'unique_value': ['Bangalore', 'West Vancouver', 'Hanover', 'Thessaloniki', 'Magdeburg']}, 'address1': {'data_type': 'StringType()', 'percentage_missing_value': 16.18, 'unique_value': ['576 Seymour Street', '1901 Munsey Drive', '651 Beacon Pkwy W. Suite 201', '1 Half Moon Ln', '30 East 23rd Street; 6th Floor